In [10]:
import pandas as pd
import re
from datetime import datetime
import numpy as np

In [11]:
year = '2020'
data = pd.read_csv('data/raw-data/wnba-raw-pbp-'+year+'.csv',)

In [12]:
data.columns

Index(['evt', 'cl', 'de', 'locX', 'locY', 'opt1', 'opt2', 'mtype', 'etype',
       'opid', 'tid', 'pid', 'hs', 'vs', 'epid', 'oftid', 'ord', 'gid', 'year',
       'gameType', 'quarter', 'date', 'homeTeam', 'awayTeam'],
      dtype='object')

In [13]:
df = data[['date','gid','gameType','homeTeam','awayTeam','quarter','cl','hs','vs','de']]
df.columns = ['Date','GameID','GameType','HomeTeam','AwayTeam','Quarter','Time','HomeScore','AwayScore','EventDescription']
if year not in ['2016','2017']:
    extracols = data[['locX','locY','opt1','opt2','mtype','etype','opid','tid','pid','epid','oftid','ord']]
else:
    extracols = data[['locX','locY','opt1','opt2','mtype','etype','opid','tid','pid','epid','oftid']]

In [14]:
#add all of the columns so they are in the same order for all years
list_of_columns = ['WinningTeam', 'Event', 'EventTeam', 'EventPlayer', 'Shooter',
       'SpecificShotType', 'ShotType', 'ShotOutcome', 'ShooterNumPTS',
       'Foul', 'FoulType', 'FoulNumFouls', 'FoulNumShots', 'TechnicalType',
       'Rebounder', 'ReboundType', 'RebounderNumOffRebounds', 'RebounderNumDefRebounds',
       'Assist', 'AssistNumAST', 'Block','BlockNumBLK', 
       'Turnover', 'TurnoverType', 'TurnoverNumTO', 'Steal', 'StealNumSTL',
       'SubstitutionOut', 'SubstitutionIn',
       'JumpBallPlayer1', 'JumpBallPlayer2', 'JumpBallPlayerPoss', 'JumpBallTeamPoss', 
       'Timeout', 'TimeoutType', 'StoppageType', 'InstantReplayType',
        'Violation','ViolationType']

for new_column in list_of_columns:
    df[new_column] = ""
    
df = df.join(extracols)

<ipython-input-14-77d28ef43c99>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column] = ""


In [15]:
df

,Date,GameID,GameType,HomeTeam,AwayTeam,Quarter,Time,HomeScore,AwayScore,EventDescription,...,opt1,opt2,mtype,etype,opid,tid,pid,epid,oftid,ord
0,2020-07-25,1022001001,regular season,NYL,SEA,1,10:00,0,0,Start Period,...,0,0,0,12,NaN,0,0,NaN,0,20000
1,2020-07-25,1022001001,regular season,NYL,SEA,1,09:52,0,0,Jump Ball Stewart vs Stokes (Loyd gains posses...,...,0,0,0,10,204329.0,1611661328,1627668,204319.0,1611661328,60000
2,2020-07-25,1022001001,regular season,NYL,SEA,1,09:48,0,0,[SEA] Stewart Jump Shot: Missed,...,2,0,1,2,NaN,1611661328,1627668,NaN,1611661328,70000
3,2020-07-25,1022001001,regular season,NYL,SEA,1,09:43,0,0,[NYL] Zahui B Rebound (Off:0 Def:1),...,0,0,0,4,NaN,1611661313,204320,NaN,1611661328,80000
4,2020-07-25,1022001001,regular season,NYL,SEA,1,09:39,0,0,[NYL] Zahui B Turnover : Bad Pass (1 TO) Steal...,...,1,0,1,5,202252.0,1611661313,204320,NaN,1611661313,90000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29935,2020-08-21,1022001075,regular season,PHO,MIN,4,00:03.0,80,87,[PHO] Diggins-Smith Technical (1 FTA),...,1,0,11,6,NaN,1611661317,203400,NaN,1611661324,5660000
29936,2020-08-21,1022001075,regular season,PHO,MIN,4,00:03.0,80,88,[MIN 88-80] Johnson Free Throw Technical (5 PTS),...,1,0,16,3,NaN,1611661324,203018,NaN,1611661324,5670000
29937,2020-08-21,1022001075,regular season,PHO,MIN,4,00:03.0,80,89,[MIN 89-80] Johnson Free Throw 1 of 2 (6 PTS),...,1,0,11,3,NaN,1611661324,203018,NaN,1611661324,5680000
29938,2020-08-21,1022001075,regular season,PHO,MIN,4,00:03.0,80,90,[MIN 90-80] Johnson Free Throw 2 of 2 (7 PTS),...,1,0,12,3,NaN,1611661324,203018,NaN,1611661324,5690000


In [16]:
#get the winners of each game
g = df.groupby(['GameID'])
d = g.last()
d = d.reset_index()
game_winners = {}
for row in range(len(d)):
    if int(d.loc[row,'HomeScore']) > int(d.loc[row,'AwayScore']):
        game_winners[d.loc[row,'GameID']] = d.loc[row,'HomeTeam']
    else:
        game_winners[d.loc[row,'GameID']] = d.loc[row,'AwayTeam'] 

In [17]:
#team ids to use for jump ball possession
team_ids = {0: np.nan,
 82: 'CHN',
 1611661313: 'NYL',
 1611661317: 'PHO',
 1611661319: 'LVA',
 1611661320: 'LAS',
 1611661321: 'DAL',
 1611661322: 'WAS',
 1611661323: 'CON',
 1611661324: 'MIN',
 1611661325: 'IND',
 1611661328: 'SEA',
 1611661329: 'CHI',
 1611661330: 'ATL'}

In [18]:
event_list = ['Rebound','Timeout','Substitution replaced by',
              '(?<!Turnover : )Foul','Turnover','Violation','Unsportsmanlike Technical',
              '(?<!Free Throw )Technical','Ejection','3pt Shot','[Ss]hot',
             'Free Throw Technical(?! [1-3])','Free Throw [1-3] of [1-3]',
               'Free Throw Flagrant [1-3] of [1-3]',
               'Free Throw Clear Path [1-3] of [1-3]',
               'Free Throw Technical [1-3] of [1-3]']
free_throws = ['Free Throw Technical','Free Throw 1 of 2','Free Throw 2 of 2',
               'Free Throw 1 of 1','Free Throw 1 of 3','Free Throw 2 of 3','Free Throw 3 of 3',
               'Free Throw Flagrant 1 of 2','Free Throw Flagrant 2 of 2', 'Free Throw Flagrant 1 of 1',
               'Free Throw Clear Path 1 of 2','Free Throw Clear Path 2 of 2',
               'Free Throw Technical 1 of 2','Free Throw Technical 2 of 2','Free Throw Technical 1 of 1']

event2_list = ['Assist','Block','Steal']

game_events = ['Period End','Period Start','Start Period','End Period',
               'Instant Replay','InstantReplay',
               'Jump Ball','Stoppage','Timeout',
               'Delay Technical','Double Technical','Too Many Players Technical',
               'Foul : Double Personal']
techs = ['Delay Technical','Double Technical','Too Many Players Technical']
event_list = '|'.join(event_list)
event2_list = '|'.join(event2_list)
game_events = '|'.join(game_events)


#def function_test(string):
#    #match = re.match(r"\[(?P<team_name>[A-Z]{3}).*\] (?P<player>\w+) (?P<specific_event>[\w\s]*)[12\:]? (?P<remaining_string>.*)", string)
#    #match = re.match(r"\[(?P<team_name>[A-Z]{3}).*\] (?P<player>[\w\s\-]*) (?P<specific_event>"+test+") [12\:\(]? (?P<remaining_string>.*)", string)
#    #match = re.match(r"\[(?P<team_name>[A-Z]{3}).*\] (?P<player>[\w\s\-]*) (?P<event>"+test+")([:\s]*)?(?P<specific_event>[\w\s\-]*)?(?P<outcome>"+outcomes+")?(?P<stat1>\(.{1,12}\))?(?P<event2>"+test2+")?([:\s]*)?(?P<player2>[\w\s\-]*)?(?P<stat2>\(.*\))?", string)
#    string = re.sub("G(\w){1,2}lich","Gulich",string)
#    print(string)
#    match = re.match(r"\[(?P<team_name>[A-Z]{3}).*\]([:\s]*)?(?P<player>[\w\s\-]*)([:\s]*)?(?P<event>"+event_list+")([:\s]*)?(?P<specific_event>[\w\s\-]*)?(?P<stat1>\(.{1,12}\))?([:\s]*)?(?P<event2>"+event2_list+")?([:\s]*)?(?P<player2>[\w\s\-]*)?(?P<stat2>\(.*\))?", string)
#    if match:
#        #print(match.group('team_name'))
#        #print(match.group('player'))
#        print(match.group('event'))
#        #print(match.group('specific_event'))
#        #print(match.group('stat1'))
#        #print(match.group('event2'))
#        #print(match.group('player2'))
#        #print(match.group('stat2'))
#        
#    else:
#        pass
#            
#            
#[function_test(str(x)) for x in df['EventDescription']]


In [19]:
#to make tests quicker
#df = df.loc[0:5000,:]

In [20]:
#shots that are too descriptive and don't parse correctly
adj_list = ['3pt','Jump','Pull-up','Pull-Up','Layup','Dunk','Driving','Cutting','Floating','Bank','Pullup','Turnaround','Step','Back','Fadeaway','Hook','Driving','Putback','Alley','Oop','Reverse','Running','Finger','Roll','Tip','No']
def append_shot_adj(player,event):
    word_list = re.split(" ",player)
    words = [word for word in word_list if word in adj_list ]
    words.append(event)
    string = " ".join(words)
    return string
def separate_player(player):
    word_list = re.split(" ",player)
    words = [word for word in word_list if word not in adj_list ]
    string = " ".join(words)
    return(string)

#an example
player = "Bird Turnaround"
event = "Jump shot"
append_shot_adj(player,event)
separate_player(player)

'Bird'

In [21]:
#test = df[df['GameID']==1011800008]
#df = test.reset_index()

In [22]:
temp_df_dts = []
temp_df_dfs = []
for row in range(len(df['EventDescription'])):
    df.loc[row,'WinningTeam'] = game_winners[df.loc[row,'GameID']]
    string = df.loc[row,'EventDescription']
    string = re.sub("G(\w){1,2}lich","Gulich",string)
    match = re.match(r"\[(?P<team_name>[A-Z]{3}).*\]([:\s]*)?(?P<player>[\w\s\-']*)([:\s]*)?(?P<event>"+event_list+")([:\s]*)?(?P<specific_event>[\w\s\-]*)?(?P<stat1>\(.{1,12}\))?([:\s]*)?(?P<event2>"+event2_list+")?([:\s]*)?(?P<player2>[\w\s\-']*)?(?P<stat2>\(.*\))?", string)
    if match:
        team = (match.group('team_name'))
        player = (match.group('player'))
        event = (match.group('event'))
        event = append_shot_adj(player,event) #fix shots that didn't parse
        player = separate_player(player) #get shooter name (remove shot adj)
        player = re.sub(' $','',player) #remove space at the end of player name
        specific_event = (match.group('specific_event'))
        specific_event = re.sub(' $','',specific_event) #remove space
        stat1 = (match.group('stat1'))
        event2 = (match.group('event2'))
        player2 = (match.group('player2'))
        player2 = re.sub(' $','',player2) #remove space
        stat2 = (match.group('stat2'))
        
        #fix blocks
        if match.group('specific_event') == 'Missed Block':
            specific_event = 'Missed'
            event2 = 'Block'
            match3 = re.match(r".*Missed Block: (?P<player2>[\w\s\-']*)(?P<stat2>\(.*\))?",string)
            player2 = (match3.group('player2'))
            stat2 = (match3.group('stat2'))
            
        #fix shot clock turnovers
        if specific_event == "Clock Turnover":
            event = 'Turnover'
            player = 'Team'
            specific_event = "Shot Clock Turnover"
            
            
        #get the stats numbers in parenthesis
        num1 = stat1
        num2 = stat2
        if stat1 is not None:
            match4 = re.match(r"[A-Za-z:\(\)\s]*(?P<number>[0-9]+)[A-Za-z:\(\)\s]*(?P<number2>[0-9]+)?",stat1)
            num1 = (match4.group('number'))
            num2 = (match4.group('number2'))
        if stat2 is not None:
            match4 = re.match(r"[A-Za-z:\(\)\s]*(?P<number>[0-9]+)[A-Za-z:\(\)\s]*(?P<number2>[0-9]+)?",stat2)
            num2 = (match4.group('number'))
        
        df.loc[row,'Event'] = event
        df.loc[row,'EventTeam'] = team_ids[df.loc[row,'tid']]
    
    #EVENT 1
        #Substitutions
        if (match.group('event')) == 'Substitution replaced by':
            df.loc[row,'Event'] = 'Substitution'
            df.loc[row,'SubstitutionOut'] = player
            player2 = specific_event
            df.loc[row,'SubstitutionIn'] = player2
        else:
            df.loc[row,'EventPlayer'] = player
        
        #Shots
        if re.match(".*[Ss]hot.*",event) or event in free_throws:
            df.loc[row,'Shooter'] = player
            df.loc[row,'SpecificShotType'] = event
            df.loc[row,'ShotType'] = '2pt Shot'
            if event == '3pt Shot':
                df.loc[row,'ShotType'] = '3pt Shot'
            if (match.group('event')) in free_throws:
                df.loc[row,'ShotType'] = 'Free Throw'
            df.loc[row,'Event'] = df.loc[row,'ShotType']
            
            df.loc[row,'ShotOutcome'] = specific_event
            #if specific_event not in ['Made','Missed']:
            #    print(specific_event)
            if (match.group('specific_event')) == '':
                df.loc[row,'ShotOutcome'] = 'Made'
                df.loc[row,'ShooterNumPTS'] = num1
        
        #Fouls
        if event == 'Foul':
            df.loc[row,'Foul'] = player
            df.loc[row,'FoulType'] = specific_event
            df.loc[row,'FoulNumFouls'] = num1
            df.loc[row,'FoulNumShots'] = num2
        
        #Turnovers
        if event == 'Turnover':
            df.loc[row,'Turnover'] = player
            df.loc[row,'TurnoverType'] = specific_event
            df.loc[row,'TurnoverNumTO'] = num1
            
        #Violations
        if event == 'Violation':
            df.loc[row,'Violation'] = player
            df.loc[row,'ViolationType'] = specific_event
            
        #Timeout
        if event == 'Timeout':
            df.loc[row,'Timeout'] = team
            df.loc[row,'TimeoutType'] = specific_event
        
        #Rebounds
        if event == 'Rebound':
            df.loc[row,'Rebounder'] = player
            if (df.loc[row,'tid']) == (df.loc[row,'oftid']): #use compare offensive team id to player team id
                df.loc[row,'ReboundType'] = 'Offensive'
            else:
                df.loc[row,'ReboundType'] = 'Defensive'
            df.loc[row,'RebounderNumOffRebounds'] = num1 #get player rebounding stats
            df.loc[row,'RebounderNumDefRebounds'] = num2
            
    #EVENT 2
        #Assists
        if event2 == 'Assist':
            df.loc[row,'Assist'] = player2
            df.loc[row,'AssistNumAST'] = num2
            
        #Blocks
        if event2 == 'Block':
            df.loc[row,'Block'] = player2
            df.loc[row,'BlockNumBLK'] = num2
            
        #Steals
        if event2 == 'Steal':
            df.loc[row,'Steal'] = player2
            df.loc[row,'StealNumSTL'] = num2
        
        
    else: #other game events that aren't player/team specific
        match2 = re.match(r"(?P<game_event>"+game_events+")?.*",string)
        event = (match2.group('game_event'))
        df.loc[row,'Event'] = event
        df.loc[row,'EventTeam'] = team_ids[df.loc[row,'tid']]
        
        #Jump Ball
        if event == 'Jump Ball':
            match3 = re.match(r"Jump Ball (?P<jb1>[\w\s\-']*) vs (?P<jb2>[\w\s\-']*)(\s?\(?(?P<jb3>[\w\s\-']*)gains possession\))?",string)
            df.loc[row,'JumpBallPlayer1'] = (match3.group('jb1'))
            df.loc[row,'JumpBallPlayer2'] = (match3.group('jb2'))
            df.loc[row,'JumpBallPlayerPoss'] = (match3.group('jb3'))
            df.loc[row,'JumpBallTeamPoss'] = team_ids[df.loc[row,'tid']] #team id is the team that gained poss
            
        #Instant Replay
        if event == 'Instant Replay' or event == 'InstantReplay':
            df.loc[row,'Event'] = 'Instant Replay'
            match3 = re.match(r"Instant[ ]?Replay[ ]?[-]? (?P<type>[\w\s\-]*)",string)
            df.loc[row,'InstantReplayType'] = (match3.group('type'))
            
        #Stoppage
        if event == 'Stoppage':
            match3 = re.match(r"Stoppage([:\s]*)?(?P<type>[\w\s\-]*)",string)
            df.loc[row,'StoppageType'] = (match3.group('type'))
            
        #Timeout
        if event == 'Timeout':
            match3 = re.match(r"Timeout([:\s]*)?(?P<type>[\w\s\-]*)",string)
            df.loc[row,'Timeout'] = (match3.group('type'))
            df.loc[row,'TimeoutType'] = (match3.group('type'))
            
        #Technical
        if event in techs:
            df.loc[row,'Event'] = 'Technical'
            df.loc[row,'TechnicalType'] = event
            if event in ['Delay Technical','Too Many Player Technical']:
                match3 = re.match(r"[\w\s']*- (?P<team>[A-Z]{3})( \((?P<fta>[0-9]) FTA\))?",string)
                df.loc[row,'EventTeam'] = (match3.group('team'))
            if event == 'Double Technical':
                match3 = re.match(r"Double Technical - (?P<p1>[\w\s\-']*), (?P<p2>[\w\s\-']*)",string)
                df.loc[row,'EventPlayer'] = (match3.group('p1'))
                df.loc[row,'TechnicalType'] = 'Double Technical'
                dtrow = df.iloc[row]
                dtrow['EventPlayer'] = match3.group('p2')
                if df.loc[row,'HomeTeam'] == df.loc[row,'EventTeam']:   
                    dtrow['EventTeam'] = df.loc[row,'AwayTeam']
                else:
                    dtrow['EventTeam'] = df.loc[row,'HomeTeam']
                temp_df_dts.append(list(dtrow))
        
        #Double Fouls
        if event == 'Foul : Double Personal':
            df.loc[row,'Event'] = 'Foul'
            df.loc[row,'FoulType'] = 'Double Personal'
            match3 = re.match(r"Foul : Double Personal - (?P<player1>[\w\s\-']*) \((?P<stat1>[0-9]) PF\), (?P<player2>[\w\s\-']*) \((?P<stat2>[0-9]) PF\)",string)
            df.loc[row,'Foul'] = match3.group('player1')
            df.loc[row,'EventPlayer'] = match3.group('player1')
            df.loc[row,'EventTeam'] = team_ids[df.loc[row,'tid']]
            df.loc[row,'FoulNumFouls'] = match3.group('stat1')
            #dfrow = df.iloc[row]
            #dfrow['Foul'] = match3.group('player2')
            #dfrow['FoulNumFouls'] = match3.group('stat2')
            #temp_df_dfs.append(list(dfrow))


<ipython-input-22-ae8232ec2ebb>:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtrow['EventPlayer'] = match3.group('p2')
<ipython-input-22-ae8232ec2ebb>:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtrow['EventTeam'] = df.loc[row,'HomeTeam']
<ipython-input-22-ae8232ec2ebb>:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtrow['EventTeam'] = df.loc[row,'AwayTeam']


In [23]:
#temp_df_dfs = pd.DataFrame(temp_df_dfs, columns=df.columns)
#df = df.append(temp_df_dfs, ignore_index=False)
#
temp_df_dts = pd.DataFrame(temp_df_dts, columns=df.columns)
df = df.append(temp_df_dts, ignore_index=False)

#df = df.sort_values('index').reset_index()
#del df['index']
#del df['level_0']

In [24]:
df = df.sort_values('Time',ascending=False)
df = df.sort_values(['GameID','Quarter'])
df = df.reset_index()
del df['index']

In [25]:
g = df.groupby(['pid','EventPlayer'])
d = g.last()
d = d.reset_index()
players = {}
for row in range(len(d)):
    players[d.loc[row,'pid']] = d.loc[row,'EventPlayer']
    if d.loc[row,'EventPlayer'] == 'Team':
        players[d.loc[row,'pid']] = d.loc[row,'EventTeam']

In [26]:
#df.to_csv('test.csv')
df.to_csv('data/wnba-pbp-'+year+'.csv')

In [30]:
df = pd.read_csv('data/wnba-pbp-'+'2016'+'.csv')
for year in range(2017,2021):
    year = str(year)
    print(year)
    temp = pd.read_csv('data/wnba-pbp-'+year+'.csv')
    df = df.append(temp)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (24,47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2017
2018
2019
2020


In [31]:
df

,Unnamed: 0,Date,GameID,GameType,HomeTeam,AwayTeam,Quarter,Time,HomeScore,AwayScore,...,opt1,opt2,mtype,etype,opid,tid,pid,epid,oftid,ord
0,0,5/14/16,1021600001,regular season,IND,DAL,1,10:00,0,0,...,0,0,0,12,NaN,0,0,NaN,0,NaN
1,1,5/14/16,1021600001,regular season,IND,DAL,1,10:00,0,0,...,0,0,0,10,201907.0,1611661325,201503,202640.0,1611661321,NaN
2,2,5/14/16,1021600001,regular season,IND,DAL,1,9:46,0,0,...,1,0,2,5,201503.0,1611661321,201907,NaN,1611661321,NaN
3,3,5/14/16,1021600001,regular season,IND,DAL,1,9:38,0,0,...,0,0,45,5,NaN,1611661325,204365,NaN,1611661325,NaN
4,4,5/14/16,1021600001,regular season,IND,DAL,1,9:26,0,0,...,3,0,1,2,NaN,1611661321,202640,NaN,1611661321,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29938,29938,2020-08-21,1022001075,regular season,PHO,MIN,4,00:03.0,80,89,...,1,0,11,3,NaN,1611661324,203018,NaN,1611661324,5680000.0
29939,29939,2020-08-21,1022001075,regular season,PHO,MIN,4,00:03.0,80,90,...,1,0,12,3,NaN,1611661324,203018,NaN,1611661324,5690000.0
29940,29940,2020-08-21,1022001075,regular season,PHO,MIN,4,00:03.0,80,87,...,1,0,11,6,NaN,1611661317,203400,NaN,1611661324,5660000.0
29941,29941,2020-08-21,1022001075,regular season,PHO,MIN,4,00:03.0,80,88,...,1,0,16,3,NaN,1611661324,203018,NaN,1611661324,5670000.0


In [33]:
df.to_csv('data/all-wnba-pbp-2016_to_8-21-2020.csv')